In [1]:
# update command line tool -> 
# (1) csv save (with more columns)
# (2) csv to model file update? (new script)
# (3) augment annotation?
import collections
import compress_pickle
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")
RESULT_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/result_files'

from AMAS import annotation_maker as am
from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [2]:
E_COLI_PATH = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS/AMAS/../tests/e_coli_core.xml'
BIOMD_190_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000190.xml')

In [3]:
recom = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)

In [4]:
print(recom.getReactionIDs())

['ODC', 'SAMdc', 'SSAT_for_S', 'SSAT_for_D', 'PAO_for_aD', 'PAO_for_aS', 'SpdS', 'SpmS', 'MAT', 'VCoA', 'VacCoA', 'P_efflux', 'aD_efflux']


In [17]:
rec_upd = pd.read_csv('species_rec.csv')
rec_upd.head()

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:15414,S-adenosyl-L-methionine,1.0,1,keep
1,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:59789,S-adenosyl-L-methionine zwitterion,1.0,0,add
2,BIOMD0000000190.xml,species,A,S-adenosylmethioninamine,metaid_0000037,CHEBI:15625,S-adenosylmethioninamine,1.0,1,delete
3,BIOMD0000000190.xml,species,P,Putrescine,metaid_0000038,CHEBI:17148,putrescine,1.0,1,keep
4,BIOMD0000000190.xml,species,P,Putrescine,metaid_0000038,CHEBI:326268,"1,4-butanediammonium",1.0,0,ignore


In [18]:
chosen = rec_upd[(rec_upd['UPDATE ANNOTATION']=='add') |\
                 (rec_upd['UPDATE ANNOTATION']=='delete')]
chosen

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
1,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:59789,S-adenosyl-L-methionine zwitterion,1.0,0,add
2,BIOMD0000000190.xml,species,A,S-adenosylmethioninamine,metaid_0000037,CHEBI:15625,S-adenosylmethioninamine,1.0,1,delete
5,BIOMD0000000190.xml,species,S,Spermine,metaid_0000039,CHEBI:15746,spermine,1.0,1,delete
23,BIOMD0000000190.xml,species,CoA,CoA,metaid_0000046,CHEBI:57287,coenzyme A(4-),1.0,0,add


In [ ]:
# case 1. add elements, there is no existing annotations
# case 2. add elements, there is existing annotations
# case 3. delete elements, there is no existing annotations
# case 4. delete elements, there is existing annotations 

In [8]:
one_type = 'species'
maker = am.AnnotationMaker(one_type)
df_type = chosen[chosen['type']==one_type]
uids = list(np.unique(df_type['id']))
meta_ids = {val:list(df_type[df_type['id']==val]['meta id'])[0] for val in uids}

NameError: name 'chosen' is not defined

In [6]:
recom.recommendReaction(ids=['ODC', 'SSAT_for_S'], min_score=0.0)

                                        ODC (credibility score: 0.817)                                       
+----+--------------+---------------+-----------------------------------------------------------------------+
|    | annotation   |   match score | label                                                                 |
+====+==============+===============+=======================================================================+
|  1 | RHEA:28827   |         1.000 | L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out) |
+----+--------------+---------------+-----------------------------------------------------------------------+
|  2 | RHEA:22964   |         0.500 | ornithine decarboxylase activity                                      |
+----+--------------+---------------+-----------------------------------------------------------------------+
|  3 | RHEA:59048   |         0.500 | D-ornithine + H(+) = CO2 + putrescine                                 |
+----+----

In [7]:
recom.selectAnnotation(('ODC', [1,2]))

Selection updated.


In [8]:
recom.selectAnnotation(('SSAT_for_S', [1]))

Selection updated.


In [9]:
recom.selection

{'species': {},
 'reaction': {'ODC':                    annotation  match score  \
  ODC (cred. 0.817)                            
  1                  RHEA:28827          1.0   
  2                  RHEA:22964          0.5   
  
                                                                 label  
  ODC (cred. 0.817)                                                     
  1                  L-ornithine(out) + putrescine(in) = L-ornithin...  
  2                                   ornithine decarboxylase activity  ,
  'SSAT_for_S':                           annotation  match score  \
  SSAT_for_S (cred. 0.985)                            
  1                         RHEA:33099          0.8   
  
                                                                        label  
  SSAT_for_S (cred. 0.985)                                                     
  1                         acetyl-CoA + spermine = CoA + H(+) + N(1)-acet...  }}

In [10]:
recom.recommendSpecies(ids=['SAM'], min_score=0.5)

                      SAM (credibility score: 0.975)                      
+----+--------------+---------------+------------------------------------+
|    | annotation   |   match score | label                              |
+====+==============+===============+====================================+
|  1 | CHEBI:15414  |         1.000 | S-adenosyl-L-methionine            |
+----+--------------+---------------+------------------------------------+
|  2 | CHEBI:59789  |         1.000 | S-adenosyl-L-methionine zwitterion |
+----+--------------+---------------+------------------------------------+



In [11]:
recom.selectAnnotation(('SAM', 2))

Selection updated.


In [12]:
recom.selection

{'species': {'SAM':                     annotation  match score  \
  SAM (cred. 0.975)                             
  2                  CHEBI:59789          1.0   
  
                                                  label  
  SAM (cred. 0.975)                                      
  2                  S-adenosyl-L-methionine zwitterion  },
 'reaction': {'ODC':                    annotation  match score  \
  ODC (cred. 0.817)                            
  1                  RHEA:28827          1.0   
  2                  RHEA:22964          0.5   
  
                                                                 label  
  ODC (cred. 0.817)                                                     
  1                  L-ornithine(out) + putrescine(in) = L-ornithin...  
  2                                   ornithine decarboxylase activity  ,
  'SSAT_for_S':                           annotation  match score  \
  SSAT_for_S (cred. 0.985)                            
  1                      

In [17]:
# new version;
model = recom.sbml_document.getModel()
TYPE_EXISTING_ATTR = {'species': recom.species.exist_annotation,
                      'reaction': recom.reactions.exist_annotation}
ELEMENT_FUNC = {'species': model.getSpecies,
                'reaction': model.getReaction}
one_type = 'reaction'

pd.set_option('display.max_colwidth', 255)
# edf: element_df
edfs = []
for one_type in recom.selection.keys():
  type_selection = recom.selection[one_type]
  for k in list(type_selection.keys()):   
    one_edf = type_selection[k]
    annotations = list(one_edf['annotation'])
    match_scores = list(one_edf['match score'])
    labels = list(one_edf['label'])
    existings = [1 if val in TYPE_EXISTING_ATTR[one_type][k] else 0 \
                 for idx, val in enumerate(one_edf['annotation'])]
    annotation2add = [val for val in TYPE_EXISTING_ATTR[one_type][k] \
                      if val not in list(one_edf['annotation'])]
    for new_anot in annotation2add:
      annotations.append(new_anot)
      if one_type=='reaction':
        match_scores.append(recom.getMatchScoreOfRHEA(k, new_anot))
        labels.append(cn.REF_RHEA2LABEL[new_anot])
      elif one_type=='species':
        match_scores.append(recom.getMatchScoreOfCHEBI(k, new_anot))
        labels.append(cn.REF_CHEBI2LABEL[new_anot])
      existings.append(1)
    new_edf = pd.DataFrame({'type': [one_type]*len(annotations),
                            'id': [k]*len(annotations),
                            'display name': [ELEMENT_FUNC[one_type](k).name]*len(annotations),
                            'meta id': [ELEMENT_FUNC[one_type](k).meta_id]*len(annotations),
                            'annotation': annotations,
                            'annotation label': labels,
                            'match score': match_scores,
                            'existing': existings})
    edfs.append(new_edf)
res = pd.concat(edfs)
res.insert(0, 'file', recom.fname)
res["USE ANNOTATION"] = 0
res

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,USE ANNOTATION
0,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:59789,S-adenosyl-L-methionine zwitterion,1.0,0,0
1,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:15414,S-adenosyl-L-methionine,1.0,1,0
0,BIOMD0000000190.xml,reaction,ODC,Ornithine decarboxylase,metaid_0000062,RHEA:28827,L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out),1.0,0,0
1,BIOMD0000000190.xml,reaction,ODC,Ornithine decarboxylase,metaid_0000062,RHEA:22964,ornithine decarboxylase activity,0.5,1,0
0,BIOMD0000000190.xml,reaction,SSAT_for_S,Spermidine/Spermine N1-acetyltransferase for S,metaid_0000064,RHEA:33099,acetyl-CoA + spermine = CoA + H(+) + N(1)-acetylspermine,0.8,1,0
1,BIOMD0000000190.xml,reaction,SSAT_for_S,Spermidine/Spermine N1-acetyltransferase for S,metaid_0000064,RHEA:28150,acetyl-CoA + spermidine = CoA + H(+) + N(1)-acetylspermidine,0.4,1,0
2,BIOMD0000000190.xml,reaction,SSAT_for_S,Spermidine/Spermine N1-acetyltransferase for S,metaid_0000064,RHEA:11116,diamine N-acetyltransferase activity,0.4,1,0
3,BIOMD0000000190.xml,reaction,SSAT_for_S,Spermidine/Spermine N1-acetyltransferase for S,metaid_0000064,RHEA:28270,acetyl-CoA + spermidine = CoA + H(+) + N(8)-acetylspermidine,0.4,1,0


In [18]:
recom.reactions.exist_annotation

{'ODC': ['RHEA:22964'],
 'SAMdc': ['RHEA:15981'],
 'SSAT_for_S': ['RHEA:28150', 'RHEA:11116', 'RHEA:28270', 'RHEA:33099'],
 'SSAT_for_D': ['RHEA:28150', 'RHEA:11116', 'RHEA:28270', 'RHEA:33099'],
 'PAO_for_aD': ['RHEA:16133', 'RHEA:25800'],
 'PAO_for_aS': ['RHEA:16133', 'RHEA:25800'],
 'SpdS': ['RHEA:12721'],
 'SpmS': ['RHEA:19973'],
 'MAT': ['RHEA:21080']}

In [19]:
recom.getReactionIDs()

['ODC',
 'SAMdc',
 'SSAT_for_S',
 'SSAT_for_D',
 'PAO_for_aD',
 'PAO_for_aS',
 'SpdS',
 'SpmS',
 'MAT',
 'VCoA',
 'VacCoA',
 'P_efflux',
 'aD_efflux']

In [20]:
for idx, val in enumerate([]):
  print("haa")

In [14]:
annotation2add

['RHEA:11116', 'RHEA:28150', 'RHEA:28270']

In [23]:
pd.concat([element_df, element_df, element_df])

,annotation,match score,label,existing
SSAT_for_S (cred. 0.985),,,,
1,RHEA:33099,0.8,acetyl-CoA + spermine = CoA + H(+) + N(1)-acet...,1
1,RHEA:33099,0.8,acetyl-CoA + spermine = CoA + H(+) + N(1)-acet...,1
1,RHEA:33099,0.8,acetyl-CoA + spermine = CoA + H(+) + N(1)-acet...,1


In [15]:
# recom.saveToCSV()

In [14]:
# RES_DIR = os.path.join(AMAS_DIR, 'res')

In [18]:
# CheBI/RHEA match score

one_chebi = 'CHEBI:15351'
one_synonyms = sa.CHEBI_LOW_SYNONYMS[one_chebi]
one_synonyms

['accoa',
 's-acetyl-coa',
 'acetyl-coa',
 'acetyl coenzyme a',
 "3'-phosphoadenosine 5'-(3-{(3r)-4-[(3-{[2-(acetylsulfanyl)ethyl]amino}-3-oxopropyl)amino]-3-hydroxy-2,2-dimethyl-4-oxobutyl} dihydrogen diphosphate)",
 's-acetyl-coenzyme a']

In [24]:
charcount_sub_df

,a,b,c,d,e,f,g,h,i,j,...,0,1,2,3,4,5,6,7,8,9
78498,0.666667,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
78499,0.534522,0.000000,0.534522,0.000000,0.267261,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
78500,0.554700,0.000000,0.554700,0.000000,0.277350,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
78501,0.384900,0.000000,0.384900,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
78502,0.222834,0.037139,0.037139,0.222834,0.259973,0.037139,0.037139,0.297113,0.222834,0.0,...,0.0,0.0,0.111417,0.222834,0.074278,0.037139,0.0,0.0,0.0,0.0
78503,0.377964,0.000000,0.377964,0.000000,0.566947,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [25]:
k

'SSAT_for_S'

In [34]:
chebi_sub_df = sa.CHEBI_DF[sa.CHEBI_DF['chebi']==one_chebi]
charcount_sub_df = sa.CHARCOUNT_DF.loc[chebi_sub_df.index, :]

one_query, name_used = recom.species.prepareCounterQuery(specs=['SAM'],
                                                         ref_cols=charcount_sub_df.columns,
                                                         use_id=True)
multi_mat = charcount_sub_df.dot(one_query)
max_val = np.round(float(multi_mat.max()), cn.ROUND_DIGITS)
max_val

0.799

In [53]:
one_rid = 'SSAT_for_S'
one_rhea = 'RHEA:33099'

specs2predict = recom.reactions.reaction_components[one_rid] 
spec_results = recom.getSpeciesListRecommendation(pred_ids=specs2predict,
                                                  update=False,
                                                  method='cdist')
pred_formulas = dict()
for one_recom in spec_results:
  chebis = [val[0] for val in one_recom.candidates]
  forms = list(set([cn.REF_CHEBI2FORMULA[k] \
         for k in chebis if k in cn.REF_CHEBI2FORMULA.keys()]))
  pred_formulas[one_recom.id] = forms


query_df = pd.DataFrame(0, 
                        index=ra.REF_MAT.columns,
                        columns=[one_rid])
for one_spec_key in pred_formulas.keys():
    one_spec = pred_formulas[one_spec_key]
    # For each one_rid, set the values 1.0
    query_df.loc[[val for val in one_spec if val in query_df.index], one_rid] = 1

sub_ref_mat = ra.REF_MAT.loc[one_rhea, :]
num_matches = int(sub_ref_mat.dot(query_df))
num_cand_elements = len(sub_ref_mat.to_numpy().nonzero()[0])

res_match_score = np.round(num_matches/num_cand_elements, cn.ROUND_DIGITS)


# pred_formulas = {val:recom.species.exist_annotation_formula[val] \
#                    for val in specs2predict \
#                    if val in self.species.exist_annotation_formula.keys()}

# r2pred_spec_formulas[one_rid] = {one_spec:inp_spec_dict[one_spec] \
#                                  for one_spec in recom.reactions.reaction_components[one_rid]}

In [54]:
res_match_score

0.8

In [51]:
num_maxpos_matches

5

In [ ]:
if inp_ids:
  one_query, name_used = recom.species.prepareCounterQuery(specs=inp_ids,
                                                           ref_cols=ref_df.columns,
                                                           use_id=True)
elif inp_strs:
  one_query, name_used = recom.species.prepareCounterQuery(specs=inp_strs,
                                                           ref_cols=ref_df.columns,
                                                           use_id=False)  
else:
  return None
multi_mat = ref_df.dot(one_query)
max_val = multi_mat.max()
result = dict()
for one_spec in one_query.c